# BING Image Search API Usage Example

### パッケージのインポート

In [ ]:
import requests
import hashlib
import os
import json
import csv

### パラメータ設定

In [ ]:
API_URL = 'https://api.cognitive.microsoft.com/bing/v7.0/images/search'
COUNT = 30
DIGEST_SIZE = 16

API_KEY = '-----------place your api key -----------'
KEYWORD = 'ラーメン'
MARKET = 'ja_JP'
SAVE_DIR = 'images/ramen'
LIST_FILE_NAME = 'image_file_list.csv'
NUM_OF_IMAGES = 100

### 保存先ディレクトリ作成

In [ ]:
save_path = SAVE_DIR

try: 
    os.makedirs(save_path)
except OSError:
    if not os.path.isdir(save_path):
        raise

### URLのハッシュから重複しないファイル名を生成

In [ ]:
m = hashlib.blake2b(digest_size=DIGEST_SIZE)

def hex_file_name(file_name):
    ext = file_name.split('.')[-1]
    if ext:
        m.update(bytearray(file_name, encoding='UTF-8'))
        return m.hexdigest() + '.' + ext
    else:
        raise Exception('No File Extension!')

### レスポンスのファイルリストからファイルをダウンロード

In [ ]:
def download_batch(image_list):
    file_list = []
    num = 0
    for entry in image_list:
        url = entry['contentUrl']
        file_name = url.split('/')[-1]
        ext = file_name.split('.')[-1]
        if ext in ['jpg', 'jpeg', 'png', 'bmp', 'gif']:
            try:
                res = requests.get(url)
            except requests.exceptions.RequestException as e:
                print(e)
                continue

            if res.status_code == requests.codes.ok:
                hex_name = hex_file_name(url)
                full_path = os.path.join(save_path, hex_name)

                with open(full_path,'wb') as f:
                    f.write(res.content)
                print('saved image...{}'.format(url))
                file_list.append([url, full_path])
                num += 1
            else:
                print("HttpError:{0}  at {1}".format(res.status_code, url))
    return file_list

### APIを使った検索とダウンロード

In [ ]:
headers = {'Ocp-Apim-Subscription-Key': API_KEY} 
params = {'q': KEYWORD, 'count': COUNT, 'offset': 0, 'mkt': MARKET}

downloaded_list = []
num_downloaded = 0
offset = 0

while num_downloaded < NUM_OF_IMAGES:
    try:
        response = requests.get(API_URL, headers=headers, params=params)
    except requests.exceptions.RequestException as e:
        print(e)
        raise
    
    data = response.json()
    if data.get('_type') == 'Images':
        file_list = download_batch(data['value'])
        downloaded_list += file_list
        num_downloaded += len(file_list)
    else:
        print(data.get('message'))
        break
    offset += COUNT
    params['offset'] = offset
        
with open(os.path.join(save_path, LIST_FILE_NAME), 'w') as f:
    writer = csv.writer(f)
    writer.writerows(downloaded_list)

print('{} images were downloaded.'.format(num_downloaded))